In [19]:
import os
import pandas as pd
import numpy as np
import torch
from torchvision.models import vgg19
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import cv2
from tqdm import tqdm


In [2]:
import datasets #same dir

In [3]:
parentDir = "C:/Users/josem/Documents/schoolWork/MQP/algonauts2023_transformers#2Leader/algonauts_2023_challenge_data/"
metaDataDir = "C:/Users/josem/Documents/schoolWork/MQP/Git/MQPCode/test/subjCocoImgData/"
device = "cuda:1" if torch.cuda.is_available() else "cpu"

parentDir = "./algonauts_2023_challenge_data/"
metaDataDir = "./subjCocoImgData/"

tsfms = transforms.Compose([
    transforms.Resize((244,244)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [22]:
trainingDataset = datasets.COCOImgWithLabel(parentDir, metaDataDir,1, tsfms)
trainDataLoader = DataLoader(trainingDataset, batch_size = 32, shuffle = False)

In [9]:
device = "cpu"

numClasses = 79
cocoVGGModel = vgg19(weights = "DEFAULT")
cocoVGGModel.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features = 25088, out_features = 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(in_features = 4096, out_features = 1024),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(in_features = 1024, out_features = numClasses),
    torch.nn.Softmax(dim = 1)
)

cocoVGGModel.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [18]:
# for data in tqdm.tqdm(trainDataLoader, desc="Training", unit="batch"):
#     img, label = data

In [21]:
optim = torch.optim.Adam(cocoVGGModel.parameters(), 0.0001)
criterion = torch.nn.CrossEntropyLoss()

epochs = 20

for epoch in range(epochs):
    print(f"Epoch {epoch}")
    avgLoss = 0
    for data in tqdm(trainDataLoader, desc="Training", unit="batch"):
        img, label = data
        img = img.to(device)
        label = label.to(device)
        optim.zero_grad()
        pred = cocoVGGModel(img)
        loss = criterion(pred, label)
        avgLoss += loss.item()
        loss.backward()
        optim.step()
    print(f"Epoch {epoch} loss: {avgLoss / len(trainDataLoader)}")

        

Epoch 0


Training:   0%|          | 0/4921 [00:00<?, ?batch/s]


RuntimeError: Expected floating point type for target with class probabilities, got Int